In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [3]:
dataset = pd.read_csv('Churn_Modelling.csv', index_col = 'RowNumber')

In [4]:
dataset.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
RowNumber,,,,,,,,,,,,,
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Features and target
X = dataset.iloc[:, 2:12]
X  # drop CustomerId and Surname

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
RowNumber,,,,,,,,,,
1,619,France,Female,42,2,0.00,1,1,1,101348.88
2,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
3,502,France,Female,42,8,159660.80,3,1,0,113931.57
4,699,France,Female,39,1,0.00,2,0,0,93826.63
5,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9996,771,France,Male,39,5,0.00,2,1,0,96270.64
9997,516,France,Male,35,10,57369.61,1,1,1,101699.77
9998,709,France,Female,36,7,0.00,1,0,1,42085.58


In [6]:
# Features and target
Y = dataset.iloc[:, 12].values  # Churn column
Y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [7]:
# Pipeline: One-hot encode categorical and scale
pipeline = Pipeline([
    ('preprocess', ColumnTransformer(
        transformers=[
            ('gender', OneHotEncoder(drop='first'), ['Gender']),
            ('geo', OneHotEncoder(drop='first'), ['Geography'])
        ],
        remainder='passthrough'
    )),
    ('scaler', StandardScaler())
])

In [8]:
X = pipeline.fit_transform(X)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [10]:
# Build ANN
classifier = Sequential()
classifier.add(Dense(6, activation='relu', input_shape=(X_train.shape[1],)))
classifier.add(Dropout(0.1))
classifier.add(Dense(6, activation='relu'))
classifier.add(Dropout(0.1))
classifier.add(Dense(1, activation='sigmoid'))

C:\Users\Dell\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [13]:
# Train ANN
history = classifier.fit(X_train, y_train, batch_size=32, epochs=100, validation_split=0.1, verbose=2)

Epoch 1/100
225/225 - 7s - 32ms/step - accuracy: 0.5962 - loss: 0.7314 - val_accuracy: 0.8025 - val_loss: 0.5653
Epoch 2/100
225/225 - 1s - 5ms/step - accuracy: 0.7911 - loss: 0.5330 - val_accuracy: 0.7950 - val_loss: 0.4845
Epoch 3/100
225/225 - 1s - 5ms/step - accuracy: 0.7931 - loss: 0.4813 - val_accuracy: 0.7962 - val_loss: 0.4457
Epoch 4/100
225/225 - 1s - 5ms/step - accuracy: 0.7974 - loss: 0.4648 - val_accuracy: 0.8037 - val_loss: 0.4275
Epoch 5/100
225/225 - 1s - 5ms/step - accuracy: 0.8039 - loss: 0.4486 - val_accuracy: 0.8188 - val_loss: 0.4128
Epoch 6/100
225/225 - 1s - 5ms/step - accuracy: 0.8082 - loss: 0.4426 - val_accuracy: 0.8238 - val_loss: 0.4047
Epoch 7/100
225/225 - 1s - 5ms/step - accuracy: 0.8104 - loss: 0.4394 - val_accuracy: 0.8313 - val_loss: 0.3994
Epoch 8/100
225/225 - 1s - 4ms/step - accuracy: 0.8168 - loss: 0.4288 - val_accuracy: 0.8350 - val_loss: 0.3929
Epoch 9/100
225/225 - 1s - 5ms/step - accuracy: 0.8160 - loss: 0.4322 - val_accuracy: 0.8388 - val_loss

In [14]:
# Predict
y_pred = (classifier.predict(X_test) > 0.5).astype(int)

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step


In [15]:
y_pred = classifier.predict(X_test)
print(y_pred[:5])

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
[[0.26458508]
 [0.20964889]
 [0.14138371]
 [0.08372349]
 [0.10392021]]


In [16]:
#Let us use confusion matrix with cutoff value as 0.5
y_pred = (y_pred > 0.5).astype(int)
print(y_pred[:5])

[[0]
 [0]
 [0]
 [0]
 [0]]


In [17]:
#Making the Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1549   46]
 [ 229  176]]


In [18]:
#Accuracy of our NN
print(((cm[0][0] + cm[1][1])* 100) / len(y_test), '% of data was classified correctly')

86.25 % of data was classified correctly
